"""
Integración de la API de Google Places para obtener los datos del top 5 lugares en el destino especificado
y asociarlos con las actividades del usuario, y luego agregar estos datos a la base de datos vectorial en Pinecone.
"""


In [7]:
import pymysql
import openai
import numpy as np
import os
import requests
import pinecone
from dotenv import load_dotenv
from openai import OpenAI

# Cargar las variables de entorno
load_dotenv(dotenv_path='.env')
db_pass = os.getenv("db_pass")
api_key_pinecone = os.getenv("apipinecone")
api_key_openai = os.getenv("apigpt_key")
api_key_google = os.getenv("google_places_key")

# Inicializar OpenAI
client = OpenAI(api_key=api_key_openai)

# Inicializar Pinecone
pinecone.init(api_key=api_key_pinecone, environment="us-east-1")
index = pinecone.Index('tuguia')

# Función para obtener las preferencias del usuario
def obtener_preferencias_usuario(user_id):
    conn = pymysql.connect(host='34.175.207.112', user='root', password=db_pass, database='travel_planner')
    try:
        cursor = conn.cursor()
        query = "SELECT activity_name, preference_level FROM user_activity_preferences WHERE user_id = %s"
        cursor.execute(query, (user_id,))
        result = cursor.fetchall()
        conn.close()
        if result:
            return result
        else:
            return None
    except Exception as e:
        print(f"Error al obtener las preferencias del usuario: {e}")
        return None

# Función para vectorizar actividades
def vectorizar_actividades_y_destino(actividades, destino):
    prompt = f"Recomienda actividades en {destino} relacionadas con las siguientes actividades: {', '.join(actividades)}."
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=prompt
    )
    return response['data'][0]['embedding']

# Función para obtener mejores actividades desde Pinecone
def obtener_mejores_actividades(destino, embeddings):
    response = index.query(
        vector=embeddings,  # Usar embeddings directamente
        top_k=5,
        include_metadata=True,
        namespace="ns1"
    )
    mejores_actividades = []
    for match in response['matches']:
        actividad = match['metadata']['Actividad']
        descripcion = match['metadata']['Descripción']
        ubicacion = match['metadata']['Ubicación']
        mejores_actividades.append((actividad, descripcion, ubicacion))
    return mejores_actividades

# Función para obtener lugares desde Google Places
def obtener_lugares_google(destino, actividad):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{actividad} en {destino}",
        "key": api_key_google,
        "type": "tourist_attraction",
        "language": "es"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get("results", [])
        return results[:5]  # Devolver los primeros 5 resultados
    else:
        print(f"Error al obtener lugares de Google Places: {response.status_code}")
        return []

# Función para agregar lugares a Pinecone
def agregar_lugares_a_pinecone(lugares, actividad, descripcion, ubicacion):
    for lugar in lugares:
        metadata = {
            "Actividad": actividad,
            "Descripción": descripcion,
            "Ubicación": ubicacion,
            "Lugar": lugar['name'],
            "Dirección": lugar['formatted_address']
        }
        vector = vectorizar_actividades_y_destino([actividad], lugar['name'])
        index.upsert(vectors=[(lugar['place_id'], vector, metadata)], namespace="ns1")

# Función para generar recomendaciones de actividades
def generar_recomendaciones(destino, user_id):
    preferencias = obtener_preferencias_usuario(user_id)
    if preferencias:
        actividades = [pref[0] for pref in preferencias]
        embeddings = vectorizar_actividades_y_destino(actividades, destino)
        recomendaciones = obtener_mejores_actividades(destino, embeddings)
        recomendaciones_con_lugares = []
        for actividad, descripcion, ubicacion in recomendaciones:
            lugares = obtener_lugares_google(destino, actividad)
            agregar_lugares_a_pinecone(lugares, actividad, descripcion, ubicacion)
            recomendaciones_con_lugares.append((actividad, descripcion, ubicacion, lugares))
        return recomendaciones_con_lugares
    else:
        print("No se encontraron preferencias para el usuario.")
        return "No se encontraron preferencias para el usuario."

# Función para probar la generación de recomendaciones
def probar_generar_recomendaciones():
    user_id = int(input("Introduce tu ID de usuario: "))  # Obtener el ID del usuario
    destino = input("Introduce tu destino: ")  # Obtener el destino
    recomendaciones = generar_recomendaciones(destino, user_id)
    
    if isinstance(recomendaciones, list):  # Verificar si se obtuvieron recomendaciones
        print("Recomendaciones basadas en tus preferencias:")
        for actividad, descripcion, ubicacion, lugares in recomendaciones:
            print(f"Actividad: {actividad}")
            print(f"Descripción: {descripcion}")
            print(f"Ubicación: {ubicacion}")
            print("Lugares recomendados:")
            for lugar in lugares:
                print(f"- {lugar['name']} ({lugar['formatted_address']})")
            print("---")
    else:
        print(recomendaciones)

# Probar la función
probar_generar_recomendaciones()


Recomendaciones basadas en tus preferencias:
Actividad: Aventura en el mar
Descripción: Vive la emoción del mar con actividades acuáticas y aventura.
Ubicación: Islas Galápagos, Ecuador
Lugares recomendados:
---
Actividad: Tener una tarde de relax
Descripción: Relájate con una tarde dedicada al descanso y el bienestar personal.
Ubicación: Nueva Zelanda
Lugares recomendados:
---
Actividad: Disfrutar de un día de bienestar en Bali
Descripción: Relájate en Bali con un día dedicado al bienestar y la tranquilidad.
Ubicación: Picos de Europa, España
Lugares recomendados:
---
Actividad: Explorar las Islas Galápagos
Descripción: Explora las Islas Galápagos y sus maravillas naturales únicas.
Ubicación: Océano Atlántico, Portugal
Lugares recomendados:
---
Actividad: Disfrutar de una tarde de café
Descripción: Relájate y disfruta de una tarde tranquila con café y buena compañía.
Ubicación: Ruta del vino en La Rioja, España
Lugares recomendados:
---
